In [ ]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
from tqdm import tqdm
import matplotlib

matplotlib.rcParams['font.sans-serif'] = ['SimHei']
matplotlib.rcParams['axes.unicode_minus'] = False

下面是异常框检测函数，判别产生的bbox是否是异常框

In [ ]:
def is_abnormally_large_box(bbox, bboxes, width_threshold=3, height_threshold=3):
    """检查当前框是否异常大"""
    x, y, w, h = bbox
    current_area = w * h  # 当前框面积

    for other_bbox in bboxes:
        if other_bbox == bbox:
            continue
        ox, oy, ow, oh = other_bbox
        other_area = ow * oh  # 其他框面积

        # 双向比较宽高比
        width_ratio_1 = ow / w  # 其他框/当前框
        # width_ratio_2 = w / ow  # 当前框/其他框
        height_ratio_1 = oh / h
        # height_ratio_2 = h / oh

        # 增加面积比较
        area_ratio = other_area / current_area if current_area > 0 else float('inf')

        # 更复杂的异常判断条件
        # if (width_ratio_1 > width_threshold or
        #     height_ratio_1 > height_threshold or
        #     area_ratio >7):  # 面积超过2倍
        #     print(f"异常检测框：当前框 {(w,h)}，比较框 {(ow,oh)}")
        #     return True
        if (width_ratio_1 > width_threshold or
            height_ratio_1 > height_threshold):  # 面积超过2倍
            print(f"异常检测框：当前框 {(w,h)}，比较框 {(ow,oh)}")
            return True
    return False

如果是异常框我们需要进一步将其内部再次检测和分割将其拆分为小框

In [ ]:
def extract_small_bboxes(image, bbox, min_area=100, max_area=2000, debug_visualization=True):
    """从异常大框中提取矩形框并可视化步骤"""
    x, y, w, h = bbox
    roi = image[y:y + h, x:x + w]

    # 确保visualization文件夹存在
    os.makedirs('visualization', exist_ok=True)

    # 灰度转换
    gray_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)

    # 高通滤波器增强
    kernel = np.array([[-1, -1, -1], [-1, 9, -1], [-1, -1, -1]])
    sharpened = cv2.filter2D(gray_roi, -1, kernel)

    # Otsu's 二值化
    _, binary = cv2.threshold(sharpened, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # 形态学操作 - 膨胀和腐蚀
    kernel = np.ones((5, 5), np.uint8)
    dilated = cv2.dilate(binary, kernel, iterations=2)
    eroded = cv2.erode(dilated, kernel, iterations=1)

    # 轮廓检测
    contours, _ = cv2.findContours(eroded, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

    # 生成独立的边界框
    rectangles = []
    for cnt in contours:
        x1, y1, w1, h1 = cv2.boundingRect(cnt)
        area = w1 * h1
        if min_area < area < max_area:
            rectangles.append((x + x1, y + y1, w1, h1))  # 存储边界框

    # 可视化步骤
    if debug_visualization:
        plt.figure(figsize=(15, 6))

        # 原始灰度图
        plt.subplot(1, 6, 1)
        plt.imshow(gray_roi, cmap='gray')
        plt.title('原始灰度图')
        plt.axis('off')

        # 高通滤波器增强
        plt.subplot(1, 6, 2)
        plt.imshow(sharpened, cmap='gray')
        plt.title('高通滤波增强')
        plt.axis('off')

        # Otsu's 二值化
        plt.subplot(1, 6, 3)
        plt.imshow(binary, cmap='gray')
        plt.title('Otsu\'s 二值化')
        plt.axis('off')

        # 形态学处理结果
        plt.subplot(1, 6, 4)
        plt.imshow(eroded, cmap='gray')
        plt.title('形态学处理')
        plt.axis('off')

        # 在原始图上绘制目标框
        contour_img = roi.copy()
        for (x1, y1, w1, h1) in rectangles:
            cv2.rectangle(contour_img, (x1, y1), (x1 + w1, y1 + h1), (255, 0, 0), 2)

        plt.subplot(1, 6, 5)
        plt.imshow(contour_img)
        plt.title('检测到的目标框')
        plt.axis('off')

        # 最终处理结果显示
        plt.subplot(1, 6, 6)
        plt.imshow(eroded)  # 显示最后的处理结果
        plt.title('最终处理结果')
        plt.axis('off')

        plt.tight_layout()
        plt.savefig(f'visualization/{len(os.listdir("visualization"))+1:03d}_complete_visualization.png')
        plt.close()

    # print(f"在异常框 {bbox} 中检测到 {len(rectangles)} 个子框")
    return rectangles


将检测到的内容我们要进行划分，精子的类别划分为五种

In [ ]:
def pixel_ratio_classification(hsvimage,bboxes):
    """对bbox进行像素比例分类的后处理函数"""
    # processimage = cv2.imread(image)
    # hsv = cv2.cvtColor(processimage, cv2.COLOR_BGR2HSV)
    # 可视化
    plt.figure(figsize=(20, 12))

    # 原始图像
    plt.subplot(1, 3, 1)
    plt.imshow(hsvimage)
    plt.title('hsvimage')
    plt.axis('off')

    # 定义深紫色颜色范围
    lower_deep_purple = np.array([150, 50, 50])
    upper_deep_purple = np.array([210, 255, 255])
    # 定义浅紫色颜色范围
    lower_light_purple = np.array([120, 40, 60])
    upper_light_purple = np.array([140, 255, 255])

    # 创建颜色掩码
    deep_purple_mask = cv2.inRange(hsvimage, lower_deep_purple, upper_deep_purple)
    light_purple_mask = cv2.inRange(hsvimage, lower_light_purple, upper_light_purple)
    plt.subplot(1, 3, 2)
    plt.imshow(deep_purple_mask,cmap='gray')
    plt.title('deep_purple_mask')
    plt.subplot(1, 3, 3)
    plt.imshow(light_purple_mask, cmap='gray')
    plt.title('light_purple_mask')

    # 保存图像
    plt.savefig(f'visualization/{len(os.listdir("visualization")) + 1:03d}_processing_steps.png')
    plt.show()

    # 存储每个bbox的像素比例信息
    bbox_pixel_stats = []

    for bbox in bboxes:
        x, y, w, h = bbox
        # roi_deep_purple = deep_purple_mask[y:y + h, x:x + w]
        roi_deep_purple = deep_purple_mask[
                          max(0, y):min(hsvimage.shape[0], y + h),
                          max(0, x):min(hsvimage.shape[1], x + w)
                          ]
        roi_light_purple = light_purple_mask[
                           max(0, y):min(hsvimage.shape[0], y + h),
                           max(0, x):min(hsvimage.shape[1], x + w)
                           ]
        # print(f'roi_deep_purple:{deep_purple_mask}')
        # roi_light_purple = light_purple_mask[y:y + h, x:x + w]

        # 计算像素数
        deep_purple_count = cv2.countNonZero(roi_deep_purple)
        # print(f'deep_purple_count:{deep_purple_count}')
        light_purple_count = cv2.countNonZero(roi_light_purple)
        # print(f'light_purple_count:{light_purple_count}')

        # 计算比例
        if deep_purple_count > 0:
            light_to_deep_purple_ratio = light_purple_count / deep_purple_count
            print(f'比例:{light_to_deep_purple_ratio}')
        elif 5 > deep_purple_count >= 0:
            light_to_deep_purple_ratio = 0.00001 #如果深紫色的区域几乎没有，说明很浅或者没有颜色的可能性大，那么把这个比例放大使其满足第五类分类条件

        bbox_pixel_stats.append({
            'bbox': bbox,
            'deep_purple_count': deep_purple_count,
            'light_purple_count': light_purple_count,
            'light_to_deep_purple_ratio': light_to_deep_purple_ratio
        })

        # 按比例排序
    bbox_pixel_stats.sort(key=lambda x: x['light_to_deep_purple_ratio'], reverse=True)

    # 分类逻辑
    color_classes = {
        '类别一': [], '类别二': [],
        '类别三': [], '类别四': [],
        '类别五': []
    }

    classified_indices = set()
    #设置颜色阈值比值区间，划分类别
    for i, stat in enumerate(bbox_pixel_stats):
        if stat['light_to_deep_purple_ratio'] > 0.4:
            color_classes['类别一'].append(stat['bbox'])
            classified_indices.add(i)
        elif 0.3 <= stat['light_to_deep_purple_ratio'] <= 0.4:
            color_classes['类别二'].append(stat['bbox'])
            classified_indices.add(i)
        elif 0.1 < stat['light_to_deep_purple_ratio'] < 0.3:
            color_classes['类别三'].append(stat['bbox'])
            classified_indices.add(i)
        # elif (stat['deep_purple_count'] > 0 and stat['light_purple_count'] == 0) or (
        #         stat['deep_purple_count'] > 0 and 0 < stat['light_purple_count'] < 2):
        #     color_classes['类别四'].append(stat['bbox'])
        #     classified_indices.add(i)
        # else:
        #     color_classes['类别五'].append(stat['bbox'])
        #     classified_indices.add(i)
        elif 0.01 < stat['light_to_deep_purple_ratio'] < 0.1:
            color_classes['类别四'].append(stat['bbox'])
            classified_indices.add(i)
        elif stat['light_to_deep_purple_ratio'] < 0.01:
            color_classes['类别五'].append(stat['bbox'])
            classified_indices.add(i)

    return color_classes, bbox_pixel_stats


检测精子的主检测函数

In [ ]:
def detect_tadpoles(inputimage,debug_vis=True):
    # 读取图像
    try:
        image = cv2.imread(inputimage)
        if image is None:
            raise ValueError("读取图像失败，返回值为 None")
    except Exception as e:
        print(f"发生错误: {e}")
    original = image.copy()

    print("正在处理图像...")
    pbar = tqdm(total=5)

    # 转换到HSV色彩空间
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # 紫色区域的HSV范围
    lower_purple = np.array([120, 40, 50])
    upper_purple = np.array([210, 255, 255])
    purple_mask = cv2.inRange(hsv, lower_purple, upper_purple)
    pbar.update(1)

    # 转换为灰度图用于深色区域检测
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, dark_mask = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY_INV)
    pbar.update(1)

    # 合并掩码
    combined_mask = cv2.bitwise_or(purple_mask, dark_mask)

    # 形态学处理
    kernel = np.ones((5, 5), np.uint8)
    open_combined_mask = cv2.morphologyEx(combined_mask, cv2.MORPH_OPEN, kernel)
    close_combined_mask = cv2.morphologyEx(open_combined_mask, cv2.MORPH_CLOSE, kernel)
    pbar.update(1)

    # 找到轮廓
    contours, _ = cv2.findContours(close_combined_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if debug_vis==True:
        # 可视化
        plt.figure(figsize=(20, 12))

        # 原始图像
        plt.subplot(2, 4, 1)
        plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        plt.title('原始图像')
        plt.axis('off')

        # HSV图像
        plt.subplot(2, 4, 2)
        plt.imshow(hsv)
        plt.title('HSV色彩空间')
        plt.axis('off')

        # 紫色区域掩码
        plt.subplot(2, 4, 3)
        plt.imshow(purple_mask)
        plt.title('紫色区域掩码')
        plt.axis('off')

        # 灰度图
        plt.subplot(2, 4, 4)
        plt.imshow(gray, cmap='gray')
        plt.title('灰度图')
        plt.axis('off')

        # 深色区域掩码
        plt.subplot(2, 4, 5)
        plt.imshow(dark_mask)
        plt.title('深色区域掩码')
        plt.axis('off')

        # 合并掩码
        plt.subplot(2, 4, 6)
        plt.imshow(combined_mask)
        plt.title('合并掩码')
        plt.axis('off')

        # 开运算后的掩码
        plt.subplot(2, 4, 7)
        plt.imshow(open_combined_mask)
        plt.title('开运算后掩码')
        plt.axis('off')

        # 闭运算后的掩码
        plt.subplot(2, 4, 8)
        plt.imshow(close_combined_mask)
        plt.title('闭运算后掩码')
        plt.axis('off')

        plt.tight_layout()
        plt.suptitle('图像处理关键步骤可视化', fontsize=16)
        plt.subplots_adjust(top=0.9)

        # 保存图像
        # plt.savefig('/visualization/image_processing_steps.png', dpi=300, bbox_inches='tight')
        plt.savefig(f'visualization/{len(os.listdir("visualization"))+1:03d}_image_processing_steps.png.png')
        plt.show()

    # 修改轮廓处理部分
    bboxes = []
    abnormal_bboxes = []  # 新增一个列表存储异常框
    # 处理异常框，进行了多次迭代处理
    max_iterations = 2  # 限制最大迭代次数
    iteration = 0

    # 收集所有有效框的面积
    all_areas = [cv2.contourArea(cnt) for cnt in contours if cv2.contourArea(cnt) > 550]
    # 计算中等大小框的范围
    if all_areas:  # 确保有有效框
        mean_area = np.mean(all_areas)
        std_area = np.std(all_areas)

        # 根据均值和标准差定义中等大小框的范围
        lower_bound = mean_area - std_area
        upper_bound = mean_area + std_area
        print(f'lower_bound:{lower_bound}')
        print(f'upper_bound:{upper_bound}')

        # 初始化标准框, 找到中等大小的框
        initialized_bbox = False  # 标志位，指示是否已添加初始框
        for cnt in contours:
            area = cv2.contourArea(cnt)
            if area > 550:
                x, y, w, h = cv2.boundingRect(cnt)
                current_area = w * h

                # 选择中等大小框进行初始化
                if lower_bound <= current_area <= upper_bound and not initialized_bbox:
                    bboxes.append((x, y, w, h))  # 添加中等大小的框
                    initialized_bbox = True
                    break  # 找到一个中等大小框后可退出循环

    if not bboxes:
        print("没有合适的中等大小框可用于初始化")
        return
    print(f'bboxes:{bboxes}')

    #初始化为全局中位数的标准框大小之后我们开始使用基于中等大小的框的范围进行异常检测
    while iteration < max_iterations:
        new_abnormal_bboxes = []
        for cnt in contours:
            area = cv2.contourArea(cnt)
            if area > 550:
                x, y, w, h = cv2.boundingRect(cnt)
                current_bbox = (x, y, w, h)
                current_area = w*h
                # 检查是否是异常框
                if is_abnormally_large_box(current_bbox, bboxes) and current_area > upper_bound:
                    new_abnormal_bboxes.append(current_bbox)
                    # 对异常框进行二次检测，使用霍夫变换
                    rectangles = extract_small_bboxes(original, current_bbox, debug_visualization=True)
                    bboxes.extend(rectangles)
                else:
                    bboxes.append(current_bbox)

                    # 如果没有新的异常框，停止循环
        if not new_abnormal_bboxes:
            break

        abnormal_bboxes.extend(new_abnormal_bboxes)

        # 重新获取轮廓，处理新的掩码
        contours, _ = cv2.findContours(combined_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        iteration += 1  # 增加迭代计数

    pbar.update(1)

    # 合并所有框进行像素比例分类
    all_bboxes = bboxes + abnormal_bboxes

    # # 检查重叠，决定哪些框显示
    # final_bboxes = []
    # for bbox in all_bboxes:
    #     x1, y1, w1, h1 = bbox
    #     is_contained = False
    #
    #     for big_bbox in abnormal_bboxes:
    #         x2, y2, w2, h2 = big_bbox
    #         # 检查当前框是否被大框包围
    #         if (x1 >= x2 and y1 >= y2) and (x1 + w1 <= x2 + w2 and y1 + h1 <= y2 + h2):
    #             is_contained = True
    #             break
    #
    #     if not is_contained:
    #         final_bboxes.append(bbox)  # 仅添加未被包围的小框
    # print(f'final_bboxes:{final_bboxes}')

    color_classes, bbox_pixel_stats = pixel_ratio_classification(original, all_bboxes)

    # 结果绘制及显示
    colors = {
        '类别一': (0, 0, 255),  # 红色
        '类别二': (0, 255, 0),  # 绿色
        '类别三': (255, 0, 0),  # 蓝色
        '类别四': (255, 255, 0),  # 黑色
        '类别五': (255, 0, 255)  # 紫色
    }

    # 绘制最终结果
    final_result = original.copy()
    #
    # for bbox in abnormal_bboxes:
    # # for bbox in final_bboxes:
    #     x, y, w, h = bbox
    #     cv2.rectangle(final_result, (x, y), (x + w, y + h), (0, 0, 0), 4)  # 白色

    # 再绘制分类后的框
    for class_name, boxes in color_classes.items():
        color = colors[class_name]
        for bbox in boxes:
            x, y, w, h = bbox
            cv2.rectangle(final_result, (x, y), (x + w, y + h), color, 2)

    pbar.update(1)
    pbar.close()

    # 创建带有图例的完整显示图
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 8),
                                   gridspec_kw={'width_ratios': [4, 1]})

    # 显示主图像
    ax1.imshow(cv2.cvtColor(final_result, cv2.COLOR_BGR2RGB))
    ax1.axis('off')
    ax1.set_title('精子检测结果', fontsize=12)

    # 创建图例
    legend_elements = []
    class_counts = []
    for class_name, boxes in color_classes.items():
        count = len(boxes)
        legend_elements.append(f'{class_name}: {count}个')
        class_counts.append(count)

    y_pos = np.arange(len(legend_elements))
    bars = ax2.barh(y_pos, class_counts,
                    color=[tuple(np.array(list(colors.values())[i])[::-1] / 255)
                           for i in range(len(colors))])

    for i, bar in enumerate(bars):
        width = bar.get_width()
        ax2.text(width, bar.get_y() + bar.get_height() / 2,
                 f'{int(width)}个',
                 ha='left', va='center', fontsize=10)

    ax2.set_yticks(y_pos)
    ax2.set_yticklabels(legend_elements)
    ax2.set_title('检测统计', fontsize=12)
    ax2.invert_yaxis()

    # 调整布局
    plt.tight_layout()

    # 保存结果
    plt.savefig('result_tadpoles_with_legend.jpg', bbox_inches='tight', dpi=300,
                facecolor='white', edgecolor='none')

    # 显示结果
    plt.show()

    return final_result, color_classes, bbox_pixel_stats


主函数入口

In [ ]:
def main():
    # 使用函数处理图片
    input_image = 'sperm.jpg'  # 替换为实际图片路径
    result, statistics, box_pixels = detect_tadpoles(input_image)

    # 打印统计信息
    print("\n检测统计结果:")
    for class_name, boxes in statistics.items():
        print(f"{class_name}: {len(boxes)}个目标")

        # 打印每个框的像素比例信息
    print("\n各检测框像素比例:")
    for stat in box_pixels:
        print(f"框 {stat['bbox']}: 浅紫色/深紫色比例 {stat['light_to_deep_purple_ratio']:.4f}")
